# **Deployment**

In [1]:
# Install required packages
!pip install flask
!pip install pyngrok
!pip install opencv-python
!pip install face-recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=a702208ed0cc6ae2a8b10b67da160c511525824500bdbd58f196b326ee420bc2
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [4]:
# Importing necessary libraries
from flask import Flask, render_template_string, request, jsonify
import cv2
import numpy as np
import face_recognition
import json
import os
import base64
from pyngrok import ngrok


# Initialize Flask app
app = Flask(__name__)

# Function to enroll face (store face encoding)
def enroll_face(image, name):
    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Find face locations and encodings
    face_locations = face_recognition.face_locations(image_rgb)
    face_encodings = face_recognition.face_encodings(image_rgb, face_locations)

    if len(face_encodings) > 0:
        # For the first face, take the encoding
        face_encoding = face_encodings[0]

        # Check if the database exists
        if os.path.exists('face_database.json'):
            with open('face_database.json', 'r') as f:
                data = json.load(f)
        else:
            data = {}

        # Check if the name already exists in the database
        if name in data:
            print(f"{name} is already enrolled. Skipping enrollment.")
            return "Name already exists."

        # Add new face encoding to the database
        data[name] = face_encoding.tolist()  # Convert numpy array to list for JSON compatibility

        # Save the updated database to file
        with open('face_database.json', 'w') as f:
            json.dump(data, f)

        return f"Face of {name} enrolled successfully."
    else:
        return "No face found in the image."

# Function to recognize face (compare encodings)
def recognize_face(image):
    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Find face locations and encodings
    face_locations = face_recognition.face_locations(image_rgb)
    face_encodings = face_recognition.face_encodings(image_rgb, face_locations)

    if len(face_encodings) > 0:
        # Load the existing database
        if os.path.exists('face_database.json'):
            with open('face_database.json', 'r') as f:
                data = json.load(f)
        else:
            return "No faces enrolled."

        # Compare the face encodings with the enrolled faces
        for name, stored_encoding in data.items():
            stored_encoding = np.array(stored_encoding)  # Convert list back to numpy array
            matches = face_recognition.compare_faces([stored_encoding], face_encodings[0])

            if matches[0]:  # If face matches
                return name
        return "Unknown"  # No match found
    else:
        return "No face detected"

# Route for the main page
@app.route('/')
def index():
    return render_template_string("""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Face Recognition</title>
        </head>
        <body>
            <h2>Face Recognition System</h2>

            <div>
                <h3>Enroll your Face</h3>
                <input type="text" id="name" placeholder="Enter your name" required>
                <button onclick="enrollFace()">Enroll Face</button>
            </div>

            <div>
                <h3>Recognize your Face</h3>
                <button onclick="recognizeFace()">Recognize Face</button>
            </div>

            <div>
                <h3>Reset Enrolled Data</h3>
                <button onclick="resetData()">Reset Enrolled Data</button>
            </div>

            <video id="video" width="300" height="200" autoplay></video>

            <script>
                // Get video stream
                const video = document.getElementById('video');
                navigator.mediaDevices.getUserMedia({ video: true })
                    .then(stream => {
                        video.srcObject = stream;
                    });

                // Capture image from video stream
                function captureImage() {
                    const canvas = document.createElement('canvas');
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    const context = canvas.getContext('2d');
                    context.drawImage(video, 0, 0, canvas.width, canvas.height);
                    return canvas.toDataURL('image/jpeg'); // Return image as base64
                }

                // Enroll face function
                function enrollFace() {
                    const name = document.getElementById('name').value;
                    if (name) {
                        const image = captureImage();
                        fetch('/enroll', {
                            method: 'POST',
                            headers: {
                                'Content-Type': 'application/json',
                            },
                            body: JSON.stringify({ image: image, name: name })
                        }).then(response => response.json())
                          .then(data => alert(data.message));
                    } else {
                        alert('Please enter your name first.');
                    }
                }

                // Recognize face function
                function recognizeFace() {
                    const image = captureImage();
                    fetch('/recognize', {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json',
                        },
                        body: JSON.stringify({ image: image })
                    }).then(response => response.json())
                      .then(data => alert(data.message));
                }

                // Reset data function
                function resetData() {
                    fetch('/reset', {
                        method: 'POST',
                    }).then(response => response.json())
                      .then(data => alert(data.message));
                }
            </script>
        </body>
        </html>
    """)

# Route for face enrollment
@app.route('/enroll', methods=['POST'])
def enroll():
    data = request.get_json()
    img_data = base64.b64decode(data['image'].split(',')[1])  # Decode the base64 image
    img_array = np.frombuffer(img_data, dtype=np.uint8)
    image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    name = data['name']

    result = enroll_face(image, name)
    return jsonify({'message': result})

# Route for face recognition
@app.route('/recognize', methods=['POST'])
def recognize():
    data = request.get_json()
    img_data = base64.b64decode(data['image'].split(',')[1])  # Decode the base64 image
    img_array = np.frombuffer(img_data, dtype=np.uint8)
    image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    result = recognize_face(image)
    return jsonify({'message': result})

# Route for resetting enrolled data
@app.route('/reset', methods=['POST'])
def reset():
    if os.path.exists('face_database.json'):
        os.remove('face_database.json')
        return jsonify({'message': 'Enrolled data has been reset. You can enroll new faces now.'})
    else:
        return jsonify({'message': 'No enrolled data found.'})

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print("ngrok tunnel URL:", public_url)

# Running the Flask app (this will keep running until stopped)
if __name__ == '__main__':
    app.run()


ngrok tunnel URL: NgrokTunnel: "https://4efe-34-125-0-132.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
